##### System Libraries

In [18]:
import os
import subprocess
import logging
import re
import copy
import time
import math
import json

##### Required Libraries

In [19]:
try:
    import scapy
except ModuleNotFoundError:
    os.system('pip install scapy')
    import scapy
try:
    import tqdm
except ModuleNotFoundError:
    os.system('pip install tqdm')
    import tqdm
try:
    import requests
except ModuleNotFoundError:
    os.system('pip install requests')
    import requests
logging.getLogger("requests").setLevel(logging.INFO)
logging.getLogger("urllib3").setLevel(logging.INFO)

import scapy.plist
from scapy.all import rdpcap
from tqdm import trange,tqdm

### Analyzer

#### Minor Function Breakdowns
---
##### Generate_Logger
Generates a logger with default parameters for logging critical components and debugging
##### Generate Self IPs
Generates a list of IPs assigned to current device according to a protocol list.
##### Remove Reserved IPs
Removes IPS that belong in the reserved ranges from the list
##### List Initializer [V2]
Helper function to shorten and simplify initializing and re-initializing lists with values
##### PCAP_Input
Takes a pcap file as input and returns and stores it in memory
##### Generate Unique IPs
Generates a list of unique IPs found in the PCAP data (excluding self IPs)
##### Query Unique IPs
Requests IP information from ip-api through requests, and stores the received data in a list
##### Generate Summary
Summarizes the data in a neater form from ip-api
##### Trace IPs
Traces the IPs and returns the output

In [20]:

class Analyzer:
    if True:
        #Initialization of the logger
        logger=logging.getLogger
        stored_logger_level=[logging.INFO]
        stored_protocol_list=["IPv4","IPv6","Gateway"]

        stored_pcap_data=[]
        stored_path=[]
        
        stored_transmission_time=[]
        
        stored_self_ips=[]
        stored_unique_ips=[]
        stored_blocked_ips=[]

        stored_queried_ips=[]
    def __init__(self,path,protocol_list=stored_protocol_list,query=True,query_amount=20,remove_reserved=True):
        if True:
            self.logger=self.Generate_Logger(filename='AnalysisCore.log')
            logging.getLogger("requests").setLevel(logging.INFO)
            self.Generate_Self_IPs(protocol_list=protocol_list)
            self.PCAP_Input(path)
            self.Generate_Unique_IPs(remove_reserved=remove_reserved)
            if query==True:
                self.Query_Unique_IPs(amount=query_amount)
    def Generate_Logger(self,filename='default_logger_name.log'): 
        logging.basicConfig(filename=filename,format='%(asctime)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level[0])
        logger=logging.getLogger()
        logger.debug('Logger initialized.')
        return logger
    def List_Initializer(self,input_list,input_param):
        if isinstance(input_param,list) or isinstance(input_param,scapy.plist.PacketList):
            input_list.clear()
            for item in input_param:
                input_list.append(item)
        else:
            input_list.clear()
            input_list.append(input_param)
    def Generate_Self_IPs(self,protocol_list=stored_protocol_list):
        self.logger.info("Generating self IPs with protocols, "+str(protocol_list))
        self_ips=[]
        process=subprocess.Popen(['ipconfig'],stdout=subprocess.PIPE,text=True,shell=True)
        for line in process.stdout:
            for protocol in protocol_list:
                if re.search(protocol,line.strip()):
                    self_ips.append(line.strip()[line.strip().find(':')+2:])
        self.List_Initializer(self.stored_self_ips,self_ips)
        self.logger.debug('Stored self IPs in stored_self_ips '+str(self_ips))
        return self_ips
    def Remove_Reserved_IPs(self,ips):
        self.logger.debug(f'Removing reserved IP ranges from list')
        dot=re.escape('.')
        patterns=[f'0{dot}0{dot}0{dot}0', f'10{dot}', f'127{dot}', f'169{dot}254{dot}', f'192{dot}0{dot}0{dot}', f'192{dot}0{dot}2{dot}', f'192{dot}18{dot}', f'192{dot}19{dot}',f'192{dot}51{dot}100{dot}', f'192{dot}88{dot}99{dot}', f'192{dot}168{dot}', f'203{dot}0{dot}113{dot}', f'255{dot}', f'256{dot}']+[f'100{dot}{x}{dot}' for x in range(64,128)]+[f'172{dot}{x}{dot}' for x in range(16,32)]+[f'{x}{dot}' for x in range(224,240)]
        #self.logger.debug(patterns)
        temp=[]
        for ip in ips:
            for pattern in patterns:
                if re.match(pattern,ip):
                    self.logger.debug(f'Pruning {ip} with {pattern}')
                    temp.append(ip)
        for ip in ips:
            if re.search('::',ip):
                self.logger.debug(f'Pruning IPv6 IP {ip}')
                temp.append(ip)
        for temp_ip in temp:
            ips.remove(temp_ip)
        self.logger.info(f'Removed {temp}')
        return ips
    def Read_JSON(self,path,name):
        self.logger.debug(f'Reading JSON from {path}')
        with open(f'{path}/{name}') as file:
            data=json.load(file)
        return data
    def Write_JSON(self,dict,path,name):
        self.logger.debug(f'Writing JSON at {path}/{name}')
        os.makedirs(path,exist_ok=True)
        try:
            with open(f'{path}/{name}.json','w') as file:
                json.dump(dict,file,indent=4)
        except OSError: # if a : (colon) is found, delete them and save, if deleted will throw error for colons
            with open(f'{path}/{name.replace(':','')}.json','w') as file:
                json.dump(dict,file,indent=4)
    def Query_IP(self,ip):
        self.logger.debug('Querying for IP '+ip)
        response=requests.get(f"http://ip-api.com/json/{ip}")
        data=response.json()
        return data
    def PCAP_Input(self,path):
        self.List_Initializer(self.stored_path,path)
        self.logger.info(f'Reading file from path, "{path}"')
        directory=path[:path.find('.')]
        if os.path.isdir(f'{directory}'): #if existing data is found
            self.logger.info(f'Found existing data at path, "{directory}"')
            files=[f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
            temp=[]
            for file in files:
                data=self.Read_JSON(directory,file)
                temp.append(data)
            self.List_Initializer(self.stored_queried_ips,temp)
            self.logger.debug(f'Found {len(temp)} files at {directory}')
        file_size=os.path.getsize(path)
        print(f'Reading PCAP Data. [{math.ceil(file_size/math.pow(1024,2))} MB]')
        pcap_data=rdpcap(path)
        self.Get_Transmission_Time(pcap_data)
        self.logger.debug('Stored '+path+' data in stored_pcap_data.')
        self.List_Initializer(self.stored_pcap_data,pcap_data)
        return pcap_data
    def Get_Transmission_Time(self,pcap_data):
        self.logger.debug(f'Calculating transmission time.')
        time=math.floor(pcap_data[-1].time-pcap_data[0].time)
        self.List_Initializer(self.stored_transmission_time,time)
        return time
    def Generate_Unique_IPs(self,pcap_data=stored_pcap_data,self_ips=stored_self_ips,remove_reserved=True):
        self.logger.info('Extracting Unique IPs from PCAP Data.')
        unique_ips=[]
        blocked_ips=self_ips.copy()
        packets=pcap_data
        for packet in tqdm(packets,desc='Analyzing packets for unique IPs'):
            flag=0
            try:
                for ip in blocked_ips:
                    if packet.payload.dst==ip:
                        flag=1
                if flag==0:
                    blocked_ips.append(packet.payload.dst)
                    unique_ips.append(packet.payload.dst)
            except AttributeError:
                broadcast_temp='256.256.256.256'
                for ip in blocked_ips:
                    if ip==broadcast_temp:
                        flag=1
                if flag==0:
                    blocked_ips.append(broadcast_temp)
                    unique_ips.append(broadcast_temp)
        if remove_reserved==True:
            self.Remove_Reserved_IPs(unique_ips)
        self.List_Initializer(self.stored_unique_ips,unique_ips)
        self.logger.debug('Stored unique IPs in stored_unique_ips.')
        return unique_ips
    def Calc_Transfer_Rates(self,pcap_data=stored_pcap_data,unique_ips=stored_unique_ips):
        self.logger.info('Calculating transfer rates per IP')
            
    def Query_Unique_IPs(self,unique_ips=stored_unique_ips,amount=20):
        self.logger.info(f'Querying unique IPs.')
        queried_unique_ips=[]
        to_be_queried=unique_ips.copy()
        if len(self.stored_queried_ips)!=0: #if data already exists, then read it
            self.logger.debug(f'Found queried data. Pruning query list')
            for query in self.stored_queried_ips:   #prune the IPs to be queried
                for ip in to_be_queried:
                    if query['query']==ip:
                        to_be_queried.remove(ip)
        if len(to_be_queried)!=0: #only if there are IPs to be queried, then query
            for ip in tqdm(to_be_queried[:min(amount,45)],desc='Querying IPs'):
                response=self.Query_IP(ip)
                queried_unique_ips.append(response)
                self.Write_JSON(response,self.stored_path[0][:self.stored_path[0].find('.')],response['query'])
        else:
            self.logger.info(f'All unique IPs are queried')
            print(f'All unique IPs are already queried')
            temp={'status':'Completed',
                  'amount':f'{len(self.stored_queried_ips)} IPs',
                  'time':f'{self.stored_transmission_time[0]} seconds'}
            self.Write_JSON(temp,self.stored_path[0][:self.stored_path[0].find('.')],'status')
        if len(self.stored_queried_ips)!=0: #if theres already data then append to it, else initialize it, if removed it will always overwrite data
            for query in queried_unique_ips:
                self.stored_queried_ips.append(query)
        else:
            self.List_Initializer(self.stored_queried_ips,queried_unique_ips)
        return queried_unique_ips
    def Generate_Summary(self,data=stored_queried_ips):
        self.logger.info(f'Summarizing data.')
        summarized=[]
        for query in tqdm(data,desc='Summarizing data'):
            temp={}
            if query['status']=='success':
                temp['Location']=f"{query['city']}, {query['regionName']}, {query['country']} | Lat: {query['lat']}, Long: {query['lon']}"
                temp['Organization']=f"{query['as']}, {query['org']}, {query['isp']}"
                temp['IP']=f"{query['query']}"
            elif query['status']=='fail':
                temp['Location']=f'Unknown [Failed]'
                temp['Organization']=f'Unknown [Failed]'
                temp['IP']=f"{query['query']}"
            summarized.append(temp)
        self.logger.debug('Summarizing completed.')
        return summarized
    def Trace_IPs(self,ips=stored_unique_ips,amount=99,hops=20,timeout=1000):
        self.logger.info(f'Tracing IPs {ips[:amount]}')
        processes=[]
        for ip in ips[:amount]:
            processes.append(subprocess.Popen(["tracert","-h",str(hops),"-w",str(timeout),ip],
                                              stdout=subprocess.PIPE,text=True,shell=True))
        time.sleep(math.ceil(hops*(timeout/1000)*3.75*(1+amount*0.001)))
        outputs_temp=[]
        for process in processes:
            outputs_temp.append(process.stdout)
        output_cleaned=[]
        for output in outputs_temp:
            temp=[]
            for line in output:
                if line.strip()!='':
                    temp.append(line.strip())
            output_cleaned.append(temp)
        return output_cleaned

In [21]:
A=Analyzer('PCAP Data/720S-Down-Spo-Misc-Idling.pcap',query=True,query_amount=45,remove_reserved=False)

Reading PCAP Data. [4 MB]


Analyzing packets for unique IPs: 100%|██████████| 10839/10839 [00:00<00:00, 26322.24it/s]


All unique IPs are already queried


In [ ]:
all_ips=A.stored_self_ips+A.stored_unique_ips
all_ips
for packet in A.stored_pcap_data:
    if packet.payload.dst==
    

['26.190.207.178',
 '26.0.0.1',
 '192.168.0.128',
 '192.168.0.1',
 '2001:0:2851:fcb0:3c0b:15d6:8c44:c5ba',
 'fe80::3c0b:15d6:8c44:c5ba%10',
 '::',
 '162.159.134.234',
 '35.186.224.45',
 '239.255.255.250',
 '51.21.215.65',
 '256.256.256.256',
 '155.133.224.23',
 '34.110.207.168',
 '35.186.227.140',
 '157.240.1.60',
 '192.168.0.255',
 '20.198.167.116',
 '104.199.241.202',
 '35.186.224.40',
 '35.186.224.22',
 '35.186.224.24',
 '104.18.32.47',
 '224.0.0.251',
 'ff02::fb',
 '72.25.64.32',
 '4.213.25.240',
 '52.178.17.2',
 '224.0.0.1',
 '142.250.196.46',
 '34.149.100.209',
 '224.0.0.252',
 '202.78.239.186',
 '255.255.255.255',
 '51.116.253.170',
 '13.61.46.223',
 '162.159.137.232',
 'ff02::1',
 '40.126.17.132',
 '162.159.128.233',
 '185.199.110.133',
 '13.107.5.93',
 '172.67.177.39',
 '20.207.73.85',
 '162.159.129.233',
 '13.107.246.58',
 '20.189.173.14',
 '142.250.182.110',
 '142.251.221.164',
 '13.61.50.43',
 '20.207.73.82',
 '45.129.229.1',
 '142.250.182.132',
 '34.107.243.93',
 '52.178.1

In [28]:
A.stored_pcap_data[0].payload.dst

'192.168.0.128'

In [24]:
A.stored_unique_ips

['162.159.134.234',
 '35.186.224.45',
 '239.255.255.250',
 '51.21.215.65',
 '256.256.256.256',
 '155.133.224.23',
 '34.110.207.168',
 '35.186.227.140',
 '157.240.1.60',
 '192.168.0.255',
 '20.198.167.116',
 '104.199.241.202',
 '35.186.224.40',
 '35.186.224.22',
 '35.186.224.24',
 '104.18.32.47',
 '224.0.0.251',
 'ff02::fb',
 '72.25.64.32',
 '4.213.25.240',
 '52.178.17.2',
 '224.0.0.1',
 '142.250.196.46',
 '34.149.100.209',
 '224.0.0.252',
 '202.78.239.186',
 '255.255.255.255',
 '51.116.253.170',
 '13.61.46.223',
 '162.159.137.232',
 'ff02::1',
 '40.126.17.132',
 '162.159.128.233',
 '185.199.110.133',
 '13.107.5.93',
 '172.67.177.39',
 '20.207.73.85',
 '162.159.129.233',
 '13.107.246.58',
 '20.189.173.14',
 '142.250.182.110',
 '142.251.221.164',
 '13.61.50.43',
 '20.207.73.82',
 '45.129.229.1',
 '142.250.182.132',
 '34.107.243.93',
 '52.178.17.234',
 '45.129.229.2',
 '203.171.247.155',
 '34.117.188.166',
 '34.110.138.217',
 '142.250.195.138',
 '142.251.220.14',
 '150.171.27.12',
 '23.21

In [25]:
summarized_trace=[]
pattern=re.escape('[')
for trace in outputs_trace:
    temp=[]
    for line in trace:
        match=re.match(r"\d",line.strip())
        if match:
            temp2=line.strip()[30:]
            if re.search(pattern,temp2):
                temp2=temp2[temp2.find('[')+1:temp2.find(']')]
            temp.append(temp2)
    summarized_trace.append(temp)
summarized_trace

NameError: name 'outputs_trace' is not defined

In [ ]:

# unique_ips=[]
# temp=[]
# ip=A.stored_unique_ips[1]
# response=requests.get(f"http://ip-api.com/json/{ip}")
# data=response.json()
# temp.clear()
# temp.append(ip)
# temp.append(data)
# unique_ips.append(temp)
# unique_ips